In [84]:
import pandas as pd
import pickle
import os
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta


from tqdm.notebook import tqdm
tqdm.pandas()
from utils import full_clean, create_subframe, create_t3_df, create_t1_df, get_parallel_edges, create_complete_df, merge_df,\
    get_parallel_edges, get_weighted_edges, get_edge_info, create_post_comment_name_id, create_t3_comment_id_dict, total_post_dict,\
        create_name_id_dict
from utils.clean_df import advanced_clean

-----
Dataframes for clean BOT

In [14]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/data/dataframe/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

In [15]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/data/dataframe/10k_sub_final.csv')
popular_bot = popular_author[popular_author.is_bot == 1]

-----
load the post and comment dataframe

In [16]:
comment_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/01-20_comments_clean.csv'
comment_df = pd.read_csv(comment_fp, low_memory = False)[['author', 'id', 'created_utc', 'link_id', 'parent_id','clean_body']]

In [17]:
comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot)

  0%|          | 0/452604 [00:00<?, ?it/s]

create the comment id dictionary

In [18]:
comment_id, level_1_comments = create_t3_comment_id_dict(comment_df)

In [19]:
level_1_comments

,author,id,created_utc,link_id,parent_id,clean_body,is_bot,all_comments
0,optionseller,fconr5l,1577836849,t3_ei96zu,t3_ei96zu,amazing,False,amazingI not in this screen shot and I dont li...
1,Wolf_Of_1337_Street,fconrdc,1577836853,t3_eiatp9,t3_eiatp9,Also nice DD btw Thanks for sharing,False,Also nice DD btw Thanks for sharingIf you actu...
2,BOOGNISH_IS_REAL,fconudk,1577836914,t3_eial7r,t3_eial7r,looks garbo,False,looks garboThey lost 89 5 million last quarter...
3,DemotivatedTurtle,fconuy0,1577836925,t3_ei8tl1,t3_ei8tl1,I've got multiple SPY calls at 314 317 and 320...,False,I've got multiple SPY calls at 314 317 and 320...
4,Juicymess,fconvmp,1577836939,t3_eiaz8e,t3_eiaz8e,I feel like this is supposed to be funny,False,I feel like this is supposed to be funnyWhy is...
...,...,...,...,...,...,...,...,...
160723,npdaly,fg508r4,1580515133,t3_ewuvf8,t3_ewuvf8,Making steamed pork dumplings tonight and drin...,False,GBoom time for the weekendfNew and gayflairfas...
160724,Ardesic53,fg509pz,1580515150,t3_ewuvf8,t3_ewuvf8,Questions for the guys across the pond from th...,False,GBoom time for the weekendfNew and gayflairfas...
160725,cloud_mode,fg50aka,1580515164,t3_ewuvf8,t3_ewuvf8,Good news s Beer virus has been upgraded to H ...,False,GBoom time for the weekendfNew and gayflairfas...
160726,iwop,fg50bry,1580515186,t3_ewx5wv,t3_ewx5wv,i bet they're nice and tendy,False,Well if you made money on tesla you're clearly...


create total post dictionary

In [20]:
post_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/posts_clean_final.csv'
post_df = pd.read_csv(post_fp, low_memory = False)[['id', 'author', 'created_utc', 'clean_title', 'clean_selftext']]

In [21]:
post_df

,id,author,created_utc,clean_title,clean_selftext
0,mhj2yd,USWD_Bank,1.617235e+09,NFT Collectable USWD United States Web Dollar,removed
1,mhj3ew,KingMacias1,1.617235e+09,We made it to Vice News you dirty apes Any pub...,NaN
2,mhj3oa,jt989898,1.617235e+09,Polygon,NaN
3,mhj3vq,KenAdamsonPrime,1.617235e+09,Ubiquiti UI gets PWND by hackers stock goes up,removed
4,mhj4o5,lexmark0331,1.617235e+09,HOFV mega squeeze,removed
...,...,...,...,...,...
1085583,ewxajz,marioz64,1.580514e+09,Man stays at home because he wasn't ill enough...,NaN
1085584,ewxctl,Compher,1.580515e+09,Found this sub and enabled options yesterday n...,NaN
1085585,ewxei7,jyuliahn1,1.580515e+09,Bears be like,removed
1085586,ewxevz,jyuliahn1,1.580515e+09,Bears be like,NaN


In [22]:
total_post_dict = total_post_dict(post_df, comment_id)

----
Test</br>
post author types: 1) 没有任何评论的 2) 只有level 1 comment的 3）non level 1 但是只是post author replies的 4）non level 1，post author replies 并且有其他人参与了 5）non level 1，post author没有参与讨论

In [24]:
# load the cleaned post dataset
post_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/posts_clean_final.csv'
selected_cols = ['created_utc', 'id', 'author', 'num_replied_comments']
post_df = pd.read_csv(post_fp)[selected_cols]

/var/folders/xd/vgr9x5c91pqf90hx6b_x1mxm0000gn/T/ipykernel_40826/1079196222.py:4: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  post_df = pd.read_csv(post_fp)[selected_cols]


In [25]:
post_df

,created_utc,id,author,num_replied_comments
0,1.617235e+09,mhj2yd,USWD_Bank,0.0
1,1.617235e+09,mhj3ew,KingMacias1,0.0
2,1.617235e+09,mhj3oa,jt989898,0.0
3,1.617235e+09,mhj3vq,KenAdamsonPrime,1.0
4,1.617235e+09,mhj4o5,lexmark0331,0.0
...,...,...,...,...
1085583,1.580514e+09,ewxajz,marioz64,0.0
1085584,1.580515e+09,ewxctl,Compher,0.0
1085585,1.580515e+09,ewxei7,jyuliahn1,0.0
1085586,1.580515e+09,ewxevz,jyuliahn1,0.0


In [26]:
def has_reply(post_df_row):
    if post_df_row['num_replied_comments'] > 0:
        return 1
    return 0

In [27]:
post_df['has_reply'] = post_df.apply(lambda row: has_reply(row), axis = 1)

/var/folders/xd/vgr9x5c91pqf90hx6b_x1mxm0000gn/T/ipykernel_40826/3340546695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_df['has_reply'] = post_df.apply(lambda row: has_reply(row), axis = 1)


In [28]:
post_df

,created_utc,id,author,num_replied_comments,has_reply
0,1.617235e+09,mhj2yd,USWD_Bank,0.0,0
1,1.617235e+09,mhj3ew,KingMacias1,0.0,0
2,1.617235e+09,mhj3oa,jt989898,0.0,0
3,1.617235e+09,mhj3vq,KenAdamsonPrime,1.0,1
4,1.617235e+09,mhj4o5,lexmark0331,0.0,0
...,...,...,...,...,...
1085583,1.580514e+09,ewxajz,marioz64,0.0,0
1085584,1.580515e+09,ewxctl,Compher,0.0,0
1085585,1.580515e+09,ewxei7,jyuliahn1,0.0,0
1085586,1.580515e+09,ewxevz,jyuliahn1,0.0,0


In [29]:
author_reply_list = list(zip(post_df.author, post_df.has_reply))
post_reply_dict = dict(zip(post_df.id, author_reply_list))

In [30]:
post_reply_dict

{'mhj2yd': ('USWD_Bank', 0),
 'mhj3ew': ('KingMacias1', 0),
 'mhj3oa': ('jt989898', 0),
 'mhj3vq': ('KenAdamsonPrime', 1),
 'mhj4o5': ('lexmark0331', 0),
 'mhj5nk': ('KenAdamsonPrime', 1),
 'mhj5ts': ('turdfurby', 0),
 'mhj5u8': ('rocketseeker', 1),
 'mhj73m': ('Huffalumpkin1', 0),
 'mhj779': ('General-Pilot-8413', 0),
 'mhj7ir': ('ingconstructionllc', 1),
 'mhjawf': ('thistlegypsy', 0),
 'mhjb4h': ('MUDFLAP202030', 1),
 'mhjbb1': ('shaquawnff', 1),
 'mhjblw': ('LoryLotus', 0),
 'mhjc0o': ('shaquawnff', 0),
 'mhjcka': ('Longjumping_Bid_4427', 0),
 'mhjczg': ('Zealousideal-Sea-961', 1),
 'mhjdnn': ('MUDFLAP202030', 0),
 'mhje2x': ('USWD_Bank', 0),
 'mhjew4': ('lizabethe87', 0),
 'mhjfnl': ('Red2g3000', 1),
 'mhjgpb': ('ercanbas', 1),
 'mhjhdu': ('Man0fG0ld', 1),
 'mhjk41': ('HelpfulTear', 1),
 'mhjk7x': ('90bills', 1),
 'mhjkg9': ('Prestigious_Quan_328', 0),
 'mhjl3h': ('90bills', 1),
 'mhjllj': ('steve8675', 0),
 'mhjmtl': ('Devin-Judin', 0),
 'mhjokb': ('vietlazed', 0),
 'mhjonl': ('a

In [51]:
# load a comment dataframe
comment_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/01-20_comments_clean.csv'
selected_cols = ['created_utc', 'link_id', 'author', 'parent_id']
comment_df = pd.read_csv(comment_fp)[selected_cols]

In [52]:
bot, comment_df = full_clean(comment_df, bot_list, popular_bot)
comment_df

  0%|          | 0/451196 [00:00<?, ?it/s]

,created_utc,link_id,author,parent_id,is_bot
0,1577836804,t3_ei988r,__TSLA__,t1_fcocbfb,False
1,1577836818,t3_ei8tl1,lerakk,t1_fcon93n,False
2,1577836849,t3_ei96zu,optionseller,t3_ei96zu,False
3,1577836853,t3_eiatp9,Wolf_Of_1337_Street,t3_eiatp9,False
4,1577836873,t3_efwy0l,G00dAndPl3nty,t1_fcazzwu,False
...,...,...,...,...,...
451191,1580515188,t3_ewuvf8,e30jawn,t1_fg4zry2,False
451192,1580515190,t3_ewuvf8,redditor1246,t3_ewuvf8,False
451193,1580515192,t3_ewuvf8,KimJongUnsTrousers,t1_fg4z8ex,False
451194,1580515194,t3_ewuvf8,RollingD,t1_fg504cz,False


In [67]:
def check_type_1(post_df, post_reply_dict, comment_df):
    type_1_author = set(post_df[post_df['has_reply'] == 0]['id'])
    
    comment_df['parents'] = comment_df.groupby(['link_id'])['author'].transform(lambda x : ' '.join(x))
    
    for index, row in comment_df.iterrows():
        parents = set(row.parents.split())
        if len(parents) == 1:
            parent_id = str(row.parent_id).split('_')[-1]
            try:
                post_author, has_reply = post_reply_dict[parent_id]
                if post_author == parents[0]:
                    type_1_author.add(parent_id)
            except: continue
    
    return type_1_author

In [74]:
all_comment = pd.DataFrame()

In [75]:
dataframe_dir = '/Users/victor/Desktop/CS/UST_summer/data/dataframe'
df_list = []
for file in os.listdir(dataframe_dir):
    if file.endswith('clean.csv'):
        df = pd.read_csv(os.path.join(dataframe_dir, file))
        df_list.append(df)

In [76]:
all_comment = pd.concat(df_list)

In [77]:
comment_bot_final, comment_df_final = full_clean(all_comment, bot_list, popular_bot, by_parent=True)

  0%|          | 0/32873468 [00:00<?, ?it/s]

In [78]:
comment_bot_final.to_csv(os.path.join(dataframe_dir, 'comments_bot_final.csv'))

In [79]:
comment_df_final.to_csv(os.path.join(dataframe_dir, 'comments_clean_final.csv'))

In [81]:
comment_name_id = create_name_id_dict(comment_df_final)
post_name_id = create_name_id_dict(post_df)

In [82]:
def get_parent(comment_df):
    parent_list = []
    for index, row in comment_df.iterrows():
        if row.parent_id.split('_')[0] == 't1':
            try:
                parent_author = comment_name_id[row.parent_id.split('_')[-1]]
                parent_list.append(parent_author)
            except:
                parent_list.append(np.nan)
        else:
            try:
                parent_author = post_name_id[row.parent_id.split('_')[-1]]
                parent_list.append(parent_author)
            except:
                parent_list.append(np.nan)
    comment_df['parent_author'] = parent_list
    return comment_df

In [85]:
comment_df_final = get_parent(comment_df_final)
comment_df_final = comment_df_final[comment_df_final['parent_author'].isna() != True]

In [86]:
comment_df_final.to_csv(os.path.join(dataframe_dir, 'comments_clean_final.csv'))